In [ ]:
# List of all industies present on Instahyre 
industries_list = ['Agriculture / Dairy', 'Airlines / Aviation / Aerospace', 'Animation / Graphic / Design', 'Apparel / Fashion', 'Architecture / Interior Design',
                  'Arts / Crafts', 'Automotive', 'Banking / Financial / Insurance', 'BPO / Outsourcing', 'Chemicals / Plastics / Rubber', 'Computer Hardware / Networking',
                  'Computer Software / IT / Internet', 'Construction / Civil Engineering', 'Consumer Electronics', 'Consumer Goods / FMCG', 'Cosmetics / Beauty',
                  'Creative / Photography', 'Defense / Government', 'Education / Training', 'Electricals', 'Events Services', 'Facilities Services', 'Food / Beverages',
                  'Freight / Logistics / Transportation', 'Furniture', 'Glass / Ceramics', 'Healthcare / Hospitals', 'Hospitality / Travel', 'Human Resources / Staffing / Recruiting',
                  'Import / Export / Trading', 'Industrial Products / Machinery', 'Legal', 'Luxury Goods / Jewellery', 'Management Consulting', 'Market Research / Analytics',
                  'Marketing / Advertising', 'Media / Entertainment', 'Medical Devices', 'Mining / Metals / Steel', 'Nonprofit / Social Services', 'Office Supplies / Equipment',
                  'Oil & Gas / Infrastructure / Energy','Other', 'Paper / Forest Products','Performing Arts','Pharma / Biotech','Political Organization','Printing / Packaging',
                  'Public Relations','Publishing','Real Estate / Property', 'Religious Institutions','Renewables / Environment','Retail','Security / Law Enforcement','Semiconductors',
                  'Shipping / Marine','Telecommunications','Textiles / Garments','Tobacco','Utilities','Wellness / Fitness / Sports','Wholesale','Wine / Spirits / Brewery']

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 100)

In [ ]:
len(industries_list)

In [ ]:
# Code for checking is the crome driver working properly or not 
from selenium import webdriver   #using selenium for automation
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import numpy as np  

options = webdriver.ChromeOptions()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome()

website = 'https://www.instahyre.com/search-jobs/'  #Instahyre search jobs webpage

driver.get(website)  

time.sleep(2)

driver.quit() 

In [ ]:
# Code for gattering the links of all the jobs there on page (based on each industry present)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
from collections import OrderedDict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome()

website = 'https://www.instahyre.com/search-jobs/'

driver.get(website)
i = 0
links = []
headings = []
industry = []
locations = []

industries = driver.find_element(By.ID, 'industries-types-selectized')

for v in industries_list:
    industries.clear()
    industries.send_keys(v)
    print(f"Industry -- {v}")
    time.sleep(2)
    industries.send_keys(Keys.ENTER)
    time.sleep(1)
    show_result = driver.find_element(By.XPATH, '//button[@class="btn btn-primary show-results"]')
    show_result.click()
    time.sleep(1)

    i = 0
    while True:
        driver.implicitly_wait(5)
        time.sleep(3)


        h1 = driver.find_elements(By.XPATH, "//div[@class='company-name ng-binding']")
        for j in h1:
            # print(j.text)
            if j==[]:
                headings.append(np.nan)
            elif j.text != '':
                headings.append(j.text.strip())
                
        # location
        driver.implicitly_wait(5)
        location = driver.find_elements(By.XPATH, "//span[@class='ng-binding ng-scope']")
        for l in location:
            if location==[]:
                locations.append(np.nan)
            else:               
                locations.append(l.text.strip())
            industry.append(v)

        # job_links with explicit wait
        try:
            links_element = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'employer-profile-opportunity')))
            for link in links_element:
                href = link.get_attribute('href')
                if link == []:
                    links.append(np.nan)
                else:
                    links.append(f"{href} -- {v}")
        except:
            pass

        links = list(OrderedDict.fromkeys(links))  #we were getting one link multiple time, so used this to just get it once (use case is in next cell)

        i += 1

        print(f"Scrolled {i} times")

        time.sleep(2)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@ng-click='nextPage()']")))
            if next_button.is_displayed():
                ActionChains(driver).move_to_element(next_button).click().perform()
            else:
                print('1')
                break

        except:
            print('2')
            break
        if not next_button.is_displayed():
            print('3')
            break
    industries = driver.find_element(By.ID, 'industries-types-selectized')
    time.sleep(1)
    industries.send_keys(Keys.CONTROL + 'a')  # Select all text
    industries.send_keys(Keys.BACKSPACE)
    time.sleep(2)
    print('**************')

# driver.quit()


In [ ]:
from collections import OrderedDict  #use-case 
list(OrderedDict.fromkeys([1,3,2,4,2,9,2,1]))

In [ ]:
len(headings), len(locations), len(links), len(industry)  #checking the len of each feature extracted

In [ ]:
# adding the list in key, value pairs (basically in a dictonary)
data = {
    'headings': headings,
    'locations': locations,
    'links': links,
    'Industry' : industry
}

In [ ]:
# making a dataframe of our extracted data
df_sample = pd.DataFrame(data)

In [ ]:
df_sample.isnull().sum()

In [ ]:
df_sample.duplicated().sum()

In [ ]:
df_sample.head(5)

In [ ]:
df_sample.shape

In [ ]:
df_sample.to_csv('623_link_with_industries.csv',index=False)

In [ ]:
links_element = driver.find_elements(By.ID, 'employer-profile-opportunity')

In [ ]:
links = []
for link in links_element:
    href = link.get_attribute('href')
    links.append(href)

In [ ]:
df = pd.read_csv("623_link_with_industries.csv")

In [ ]:
df.head()

In [ ]:
df[['links','Industryy']]=df['links'].str.split(' -- ', expand=True)

In [ ]:
df['Industry'] == df['Industryy']

In [ ]:
df

In [ ]:
# don't use this code again and again
# other wise it will ban the website

In [ ]:
# webscarpping code to get other details
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

location = []
company_names = []
estab_years = []
employees_count = []
job_id = []
skill_list = []
hr_name = []
designation = [] 
involvement = []
links_ = []
experience = []

count = -1

# Set up the Selenium webdriver
driver = webdriver.Chrome()

for i in df['links']:
    try:
        url = i
        driver.get(url)
        
        time.sleep(3)
        driver.implicitly_wait(5)
        # Location
        location_element = driver.find_element(By.XPATH, '//span[@class="ng-binding"]')
        if location_element == []:
            location.append(np.nan)
        else:
            location.append(location_element.text.strip())

        # Company Name
        driver.implicitly_wait(5)
        company_name_element = driver.find_element(By.XPATH, '//h2[@class="company-name ng-binding"]')
        if company_name_element == []:
            company_names.append(np.nan)
        else:
            company_names.append(company_name_element.text.strip())

        # Establishment Year
        driver.implicitly_wait(5)
        estab_year_element = driver.find_element(By.XPATH, '//div[@ng-if="employer.company_founded"]')
        if estab_year_element == []:
            estab_years.append(np.nan)
        else:
            estab_years.append(estab_year_element.text.strip())

            
        # Employees Count
        driver.implicitly_wait(5)
        employees_count_element = driver.find_element(By.XPATH, '//div[@ng-if="employer.employee_count"]')
        if employees_count_element == []:
            employees_count.append(np.nan)
        else:
            employees_count.append(employees_count_element.text.strip())
            
        #job_id
        driver.implicitly_wait(5)
        job_id.append(int(url.split('-')[1]))

        #skills
        driver.implicitly_wait(5)
        skills_element = driver.find_elements(By.ID, 'job-skills-description')
        if skills_element == []:
            skill_list.append(np.nan)
        else:
            for j in skills_element:
                skill_list.append(j.text.replace('\n',', '))

        #hr-name
        driver.implicitly_wait(5)
        hr_name_element = driver.find_element(By.XPATH, '//span[@class="rec-name"]')
        if hr_name_element == []:
            hr_name.append(np.nan)
        else:
            hr_name.append(hr_name_element.text.strip())

        #designation
        driver.implicitly_wait(5)
        designation_element = driver.find_element(By.XPATH, '//h1[@class="ng-binding"]')
        if designation_element == []:
            designation.append(np.nan)  
        else:
            designation.append(designation_element.text.strip())

        #involvement
        driver.implicitly_wait(5)
        involvement_element = driver.find_element(By.XPATH, '//p[@class="ng-binding"]')
        if involvement_element == []:
            involvement.append(np.nan)  
        else:
            involvement.append(involvement_element.text.strip('Function: '))

        
        #exp
        driver.implicitly_wait(5)
        exp_element = driver.find_element(By.XPATH, "//span[@class='experience ng-binding ng-scope']")
        if exp_element == []:
            experience.append(np.nan)
        else:
            experience.append(exp_element.text.strip())
        
        
        #job_link
        driver.implicitly_wait(5)
        links_.append(url)

        

        count += 1
        print(count, i)
    except Exception as e:
        print(f"Error occurred for {i}: {str(e)}")
        
        driver.quit()

# Close the browser


print(count, "end")

In [ ]:
# location = []
# company_names = []
# estab_years = []
# employees_count = []
# job_id = []
# skill_list = []
# hr_name = []
# designation = [] 
# involvement = []
# links_ = []
# experience = []
print(f"Lenght of location: {len(location)}"), 
print(f"Lenght of company_names: {len(company_names)}")
print(f"Lenght of estab_years: {len(estab_years)}")
print(f"Lenght of employees_count: {len(employees_count)}")
print(f"Lenght of job_id: {len(job_id)}")
print(f"Lenght of skill_list: {len(skill_list)}")
print(f"Lenght of hr_name: {len(hr_name)}")
print(f"Lenght of designation: {len(designation)}")
print(f"Lenght of involvement: {len(involvement)}")
print(f"Lenght of links_: {len(links_)}")
print(f"Lenght of experience: {len(experience)}")

In [ ]:
data_2 = {
    'name': company_names,
'estab_year': estab_years,
'employees_count': employees_count,
    'location' : location,
    'job_ids' : job_id,
    'skills' : skill_list,
    'hr' : hr_name,
    'Designation' : designation,
    'Involvement' : involvement,
    'Exp' : experience,
    'url' : links_
}

In [ ]:
df_sample2 = pd.DataFrame(data_2)

In [ ]:
df_sample2

In [ ]:
df_sample2.to_csv('final_623_data.csv',index=False)

In [ ]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

df_sample2.style.format(make_clickable)

In [ ]:
from selenium import webdriver   #using selenium for automation
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import numpy as np
import pandas as pd  
import re

skill = []

options = webdriver.ChromeOptions()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome()

website = 'https://www.instahyre.com/job-266464-senior-product-designer-at-syndr-singapore/' 

driver.get(website)

time.sleep(2)

skills = driver.find_elements(By.ID, 'job-skills-description')

if skills == []:
    skill.append(np.nan)
else:
    for j in skills:
        skill.append(j.text.replace('\n',', '))

In [ ]:
skill

In [ ]:
len(skill)

In [ ]:
from selenium import webdriver   #using selenium for automation
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import numpy as np
import pandas as pd  
import re

skill = []

options = webdriver.ChromeOptions()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome()

website = 'https://www.instahyre.com/job-266323-sales-manager-portfolio-drive-at-axis-bank-amritsar-chennai-delhi/' 

driver.get(website)

time.sleep(2)

skills = driver.find_elements(By.ID, 'job-skills-description')

if skills == []:
    skill.append(None)
else: 
    for j in skills:
        skill.append(j.text.replace('\n',', '))

In [ ]:
skill

In [ ]:
df_2 = pd.read_csv("final_623_data.csv")

In [ ]:
df_2

In [ ]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

df_2.style.format(make_clickable)

In [ ]:
df_2.isnull().sum()

In [ ]:
df_2[df_2['name'].isnull()==True]

In [ ]:
df_2[df_2['employees_count'].isnull()==True]

In [ ]:
df.head(1)

In [ ]:
df_3 = pd.merge(df_2, df, how='right', left_on='url', right_on='links')

In [ ]:
df_3

In [ ]:
df_3.duplicated().sum()

In [ ]:
df_3.isnull().sum()

In [ ]:
df_3.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
df_3

In [ ]:
df_3.dropna(axis=0,inplace=True)

In [ ]:
df_3.shape

In [ ]:
df_3.drop(['headings','locations', 'links','Industryy'],axis=1, inplace=True)

In [ ]:
df_3

In [ ]:
df_3.shape

In [ ]:
df_3.to_csv("Table_622.csv",index=False)

In [ ]:
df_new = pd.read_csv("Table_622.csv")

In [ ]:
df_new